# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [42]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [43]:
#Load the csv exported in Part I to a DataFrame
city_weather_df = pd.read_csv("cities.csv")
city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ancud,-41.8697,-73.8203,55.99,100,90,21.85,CL,1619838467
1,butaritari,3.0707,172.7902,83.34,72,68,8.50,KI,1619838047
2,east london,65.0000,-15.0000,27.43,91,100,8.93,IS,1619838551
3,encruzilhada do sul,-15.5314,-40.9094,65.34,89,98,8.84,BR,1619838551
4,victoria,22.2855,114.1577,82.00,69,98,3.87,HK,1619838408
...,...,...,...,...,...,...,...,...,...
549,liwale,-9.7667,37.9333,67.10,97,54,2.35,TZ,1619838702
550,sapa,5.1544,120.3241,85.87,66,93,7.61,PH,1619838703
551,comodoro rivadavia,-13.6631,-59.7858,65.57,79,0,3.60,BR,1619838703
552,manokwari,-0.8667,134.0833,82.92,73,97,2.91,ID,1619838520


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [44]:
#configure gmaps
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations
locations = city_weather_df[["Lat","Lng"]]

#Add Heatmap layer to map
humidity = city_weather_df["Humidity"]

In [45]:
#Plot heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [46]:
smaller_city_df = city_weather_df.loc[(city_weather_df["Wind Speed"] <= 10) & \
                  (city_weather_df["Max Temp"] >= 70) & (city_weather_df["Max Temp"] <= 80)].dropna()
smaller_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,billings,45.7833,-108.5007,75.20,25,75,5.75,US,1619837810
18,saint-philippe,-21.3585,55.7679,73.40,78,0,6.91,RE,1619838380
22,atar,20.5169,-13.0499,79.11,20,0,6.60,MR,1619838557
24,korla,41.7597,86.1469,77.11,11,0,6.08,CN,1619838558
45,port alfred,-7.0349,29.7638,71.38,82,100,3.36,CD,1619838563
...,...,...,...,...,...,...,...,...,...
523,saint-marc,19.1082,-72.6938,78.78,70,19,2.68,HT,1619838695
532,conde,-7.2597,-34.9075,75.20,94,75,5.75,BR,1619838697
533,aracati,-4.5617,-37.7697,76.01,86,98,8.97,BR,1619838697
542,mayumba,-3.4320,10.6554,77.95,88,100,4.72,GA,1619838700


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [47]:
#Store into variable named hotel_df
hotel_df = smaller_city_df.loc[:,["City","Country","Lat","Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
6,billings,US,45.7833,-108.5007,
18,saint-philippe,RE,-21.3585,55.7679,
22,atar,MR,20.5169,-13.0499,
24,korla,CN,41.7597,86.1469,
45,port alfred,CD,-7.0349,29.7638,
...,...,...,...,...,...
523,saint-marc,HT,19.1082,-72.6938,
532,conde,BR,-7.2597,-34.9075,
533,aracati,BR,-4.5617,-37.7697,
542,mayumba,GA,-3.4320,10.6554,


In [68]:
 #Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 1000000,
          "key" : g_key
}

In [69]:
 for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
    print("--------------------Ending Search")

Retrieving Results for Index 6: billings.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 18: saint-philippe.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 22: atar.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 24: korla.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 45: port alfred.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 51: half moon bay.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 57: new norfolk.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 61: tabou.
Could not find result
-------------------------
--------------------Ending S

Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 426: tallahassee.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 432: douglas.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 437: mangan.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 445: gamba.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 455: lahij.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 457: cap-haitien.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 459: bull savanna.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 463: m

In [65]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
6,billings,US,45.7833,-108.5007,
18,saint-philippe,RE,-21.3585,55.7679,
22,atar,MR,20.5169,-13.0499,
24,korla,CN,41.7597,86.1469,
45,port alfred,CD,-7.0349,29.7638,
...,...,...,...,...,...
523,saint-marc,HT,19.1082,-72.6938,
532,conde,BR,-7.2597,-34.9075,
533,aracati,BR,-4.5617,-37.7697,
542,mayumba,GA,-3.4320,10.6554,


In [51]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [52]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))